Dependencias

In [1]:
# Instalar nbconvert si no está disponible
!pip install nbconvert


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import pandas as pd
import joblib
import os

In [3]:
possible_paths = [
    '../analisis_exploratorios_tp3/',  
    '../../analisis_exploratorios_tp3/' 
]

data_path = None
for path in possible_paths:
    if os.path.exists(f"{path}train.csv"):
        data_path = path
        break

if data_path:
    print(f"Archivos CSV encontrados en: {data_path}")
    train = pd.read_csv(f'{data_path}train.csv', index_col=0, parse_dates=True)
    validation = pd.read_csv(f'{data_path}validation.csv', index_col=0, parse_dates=True)
    test = pd.read_csv(f'{data_path}test.csv', index_col=0, parse_dates=True)
else:
    print("❌ No se encontraron los archivos CSV en ninguna ruta")

print(f"Datos cargados exitosamente:")
print(f"   Train: {train.shape}")
print(f"   Validation: {validation.shape}")
print(f"   Test: {test.shape}")
print(f"   Columnas disponibles: {len(train.columns)} columnas")

Archivos CSV encontrados en: ../analisis_exploratorios_tp3/
Datos cargados exitosamente:
   Train: (1110, 48)
   Validation: (370, 48)
   Test: (370, 48)
   Columnas disponibles: 48 columnas


In [7]:
try:
    lr_models = joblib.load('../experimentostp3/src/models/ridge_models_optimizados.joblib')
except Exception as e:
    print(f" Error cargando modelos RL: {e}")
    print(" Verificando archivos disponibles...")
    import os
    lr_path = '../experimentostp3/modelos_entrenados/'
    if os.path.exists(lr_path):
        files = os.listdir(lr_path)
        print(f"   Archivos encontrados: {files}")
    else:
        print(f"   Directorio no encontrado: {lr_path}")

btc_completo = pd.concat([train, validation, test]).sort_index()
latest_data = btc_completo.iloc[-1:].copy()
last_known_date = latest_data.index[0]

future_predictions_lr = {}
targets = [f"NextClose_BTC{i}" for i in range(1, 8)]

for target in targets:
    try:
        model = lr_models[target]
        input_data = latest_data.copy()
        if input_data.isnull().values.any():
            input_data = input_data.fillna(method='ffill').fillna(method='bfill')
        
        prediction = model.predict(input_data)[0]
        future_predictions_lr[target] = prediction
        
    except Exception as e:
        print(f"   Error en {target}: {e}")

if len(future_predictions_lr) >= 1:
    first_target = "NextClose_BTC1"
    if first_target in future_predictions_lr:
        first_date = last_known_date + pd.Timedelta(days=1)
        first_prediction = future_predictions_lr[first_target]
        print(" PREDICCIÓN REGRESIÓN LINEAL - DÍA 1")
        print(f"Fecha: {first_date.strftime('%Y-%m-%d (%A)')}")
        print(f"Predicción RL: ${first_prediction:,.2f}")
    else:
        print(f"\n No se generó predicción para {first_target}")
else:
    print(f"\n No se generaron predicciones de Regresión Lineal")

 PREDICCIÓN REGRESIÓN LINEAL - DÍA 1
Fecha: 2025-10-24 (Friday)
Predicción RL: $110,050.82


/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_74727/2180660162.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill').fillna(method='bfill')
/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_74727/2180660162.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill').fillna(method='bfill')
/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_74727/2180660162.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill').fillna(method='bfill')
/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_74727/2180660162.py:26: FutureWarning: DataFrame.fillna with 'method

In [9]:
try:
    rf_models = joblib.load('../experimentostp3/modelos_entrenados/rf_modelos_optimizados.joblib')
    features_por_target = joblib.load('../experimentostp3/modelos_entrenados/rf_features_por_target.joblib')
    
except Exception as e:
    print(f" Error cargando modelos RF optimizados: {e}")
    
    try:
        rf_models = joblib.load('../experimentostp3/modelos_entrenados.joblib')
        features_por_target = joblib.load('../experimentostp3/features_por_target.joblib')
    except:
        print(" No se encontraron modelos Random Forest")
        rf_models = None


if rf_models is not None:
    btc_completo = pd.concat([train, validation, test]).sort_index()
    latest_data = btc_completo.iloc[-1:].copy()
    last_known_date = latest_data.index[0]
    

    future_predictions = {}
    error_messages = []
    
    print(f"\nGenerando predicciones RF OPTIMIZADAS desde: {last_known_date.strftime('%Y-%m-%d')}")
    print(f" Precio actual de referencia: ${latest_data['btc_c'].iloc[0]:,.2f}")
    
    for target in targets:
        try:
            model = rf_models[target]
            
            input_data = latest_data.copy()
            if input_data.isnull().values.any():
                input_data = input_data.fillna(method='ffill').fillna(method='bfill')
            
            prediction = model.predict(input_data)[0]
            future_predictions[target] = prediction
            
        except Exception as e:
            error_msg = f" Error en {target}: {str(e)}"
            error_messages.append(error_msg)
    

    if len(future_predictions) >= 6:  
       
        filtered_predictions = {k: v for k, v in future_predictions.items() if k != "NextClose_BTC1"}
        
        if len(filtered_predictions) >= 1:
           
            prediction_dates = pd.date_range(start=last_known_date + pd.Timedelta(days=2), 
                                           periods=len(filtered_predictions))
            
            results_rf_df = pd.DataFrame({
                'Fecha': prediction_dates,
                'Target': list(filtered_predictions.keys()),
                'Predicción RF Optimizado (USD)': list(filtered_predictions.values())
            })
            results_rf_df['Fecha'] = results_rf_df['Fecha'].dt.strftime('%Y-%m-%d (%A)')
            results_rf_df['Predicción RF Optimizado (USD)'] = results_rf_df['Predicción RF Optimizado (USD)'].map('${:,.2f}'.format)

            print(" PREDICCIONES RANDOM FOREST OPTIMIZADO - DÍAS 2-7")
            print(results_rf_df.to_string(index=False))
        else:
            print(f"\n No hay predicciones válidas después de excluir BTC1")
        
        if error_messages:
            print("\n Errores encontrados:")
            for msg in error_messages:
                print(f"   {msg}")
        
    else:
        print(f"\n Solo se generaron {len(future_predictions)} predicciones RF")
        print("Se necesitan al menos 6 predicciones para mostrar días 2-7")
        print("Errores encontrados:")
        for msg in error_messages:
            print(f"  - {msg}")


Generando predicciones RF OPTIMIZADAS desde: 2025-10-23
 Precio actual de referencia: $110,069.73
 PREDICCIONES RANDOM FOREST OPTIMIZADO - DÍAS 2-7
                 Fecha         Target Predicción RF Optimizado (USD)
 2025-10-25 (Saturday) NextClose_BTC2                    $110,113.80
   2025-10-26 (Sunday) NextClose_BTC3                    $110,789.55
   2025-10-27 (Monday) NextClose_BTC4                    $109,725.55
  2025-10-28 (Tuesday) NextClose_BTC5                    $109,657.40
2025-10-29 (Wednesday) NextClose_BTC6                    $109,890.39
 2025-10-30 (Thursday) NextClose_BTC7                    $108,528.58


/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_74727/4173821441.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill').fillna(method='bfill')
/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_74727/4173821441.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill').fillna(method='bfill')
/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_74727/4173821441.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  input_data = input_data.fillna(method='ffill').fillna(method='bfill')
/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_74727/4173821441.py:34: FutureWarning: DataFrame.fillna with 'method